# Histogram of Oriented gradients
The histogram of oriented gradients (HOG) is a feature descriptor used in computer vision and image processing for the purpose of object detection. The technique counts occurrences of gradient orientation in localized portions of an image. 

In [1]:
import numpy as np
import cv2
from skimage import data, color, feature
import skimage.data
from sklearn.datasets import fetch_lfw_people
from skimage import data, transform
from sklearn.feature_extraction.image import PatchExtractor
from itertools import chain
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
print("imported")



imported


# LFWdataset:
Loader for the Labeled Faces in the Wild (LFW) people dataset

This dataset is a collection of JPEG pictures of famous people collected on the internet

In [3]:
faces = fetch_lfw_people()
positive_patches = faces.images
positive_patches.shape


(13233, 62, 47)

# Negative Images:
Get random images that DO NOT have a face in it

In [5]:
imgs_to_use = ['camera', 'text', 'coins', 'moon',
               'page', 'clock', 'immunohistochemistry',
               'chelsea', 'coffee', 'hubble_deep_field']
images = [color.rgb2gray(getattr(data, name)())
          for name in imgs_to_use]
print('done')

done


In [6]:
def extract_patches(img, N, scale=1.0, patch_size=positive_patches[0].shape):
    extracted_patch_size = tuple((scale * np.array(patch_size)).astype(int))
    extractor = PatchExtractor(patch_size=extracted_patch_size,
                               max_patches=N, random_state=0)
    patches = extractor.transform(img[np.newaxis])
    if scale != 1:
        patches = np.array([transform.resize(patch, patch_size)
                            for patch in patches])
    return patches

negative_patches = np.vstack([extract_patches(im, 1000, scale)
                              for im in images for scale in [0.5, 1.0, 2.0]])
negative_patches.shape

(30000, 62, 47)

# SVM Classification:
Now we are going to train a classifier on both these patches.
Using LinearSVC for classification.

In [7]:
X_train = np.array([feature.hog(im)
                    for im in chain(positive_patches,
                                    negative_patches)])
y_train = np.zeros(X_train.shape[0])
y_train[:positive_patches.shape[0]] = 1
X_train.shape


(43233, 1215)

In [8]:
model = LinearSVC(C=0.4)

In [9]:
model.fit(X_train, y_train)


LinearSVC(C=0.4, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [10]:
cross_val_score(LinearSVC(C=0.4), X_train, y_train)


array([0.99005435, 0.98496588, 0.99329247, 0.98612075, 0.99097849])

# Real Time Video Detection

Design a Sliding Window that ll extract a patch from the image to classify it

In [11]:
def sliding_window(img, patch_size=positive_patches[0].shape,
                   istep=2, jstep=2, scale=1.0):
    Ni, Nj = (int(scale * s) for s in patch_size)
    for i in range(0, img.shape[0] - Ni, istep):
        for j in range(0, img.shape[1] - Ni, jstep):
            patch = img[i:i + Ni, j:j + Nj]
            if scale != 1:
                patch = transform.resize(patch, patch_size)
            yield (i, j), patch
            
Ni, Nj = positive_patches[0].shape


In [ ]:
cap = cv2.VideoCapture(0)

while True:
    _,frame = cap.read()

    frame = skimage.color.rgb2gray(frame)
    frame = skimage.transform.rescale(frame, 0.5)
    
    indices, patches = zip(*sliding_window(frame))
    
    #  Generate hog for each patch from sliding window
    
    patches_hog = np.array([feature.hog(patch) for patch in patches])
    labels = model.predict(patches_hog)
    
    Ni, Nj = positive_patches[0].shape
    indices = np.array(indices)

    # draw rectangles for patches which have label as 1(human face detected)
    
    for i, j in indices[labels == 1]:
        cv2.rectangle(frame, (j,i),(j+Nj,i+Ni),(255,0,0),3)

    
    cv2.imshow("frame",frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

    
    
    
    
    

# comments ::
            1. FPSislow
            2. Non msxsuppression has to be done
            3.Many false positives